### Container Object
Container Objects adalah obyek yang mempunyai sifat sebagai penampung, berisi lebih dari satu obyek.
Source: https://spacy.io/api

### Container Objects 1 - Language
Source: https://spacy.io/api/language

In [15]:
# Definisi language - menggunakan spacy.load
import spacy
import itertools

nlp = spacy.load("en_core_web_sm")
# --- sampai di sini ----

print(nlp)

print(nlp.vocab.strings)

print(len(nlp.vocab.strings))
words = set(nlp.vocab.strings)
word = 'speech'
print(f"Is '{word}' an English word?: {word in words}")
word = 'siapa'
print("Is '" ,word, "' an English word?:", word in words)

for i, val in enumerate(itertools.islice(words, 10)):
    print(i, val)

83839
Is 'speech' an English word?: True
Is ' siapa ' an English word?: False
0 Aznar
1 Jonathan
2 Hardest
3 flawlessly
4 Cocom
5 Liberation
6 Jonson
7 frameworks
8 ashes
9 alisky


In [17]:
# Definisi language - cara lain
import spacy

from spacy.lang.en import English
nlp = English()
print(nlp)
print(nlp.vocab.strings)

In [18]:
# Jika ingin menggunakan API untuk language - vocab
# tanpa definisi bahasa tertentu
# (membangun dari awal)

from spacy.vocab import Vocab
from spacy.language import Language
nlp = Language(Vocab())
print(nlp)

'''
Penting: kelas Language akan memproses teks dan menghasilkan obyek Doc. 
Doc merupakan container objects. 
Kita akan mempelajari sedikit demi sedikit dari arsitektur tersebut, 
dimulai dari container objects.
'''

'\nPenting: kelas Language akan memproses teks dan menghasilkan obyek Doc. \nDoc merupakan container objects. \nKita akan mempelajari sedikit demi sedikit dari arsitektur tersebut, \ndimulai dari container objects.\n'

### Container Objects 2 - Doc
Lengkap: https://spacy.io/api/doc

Merupakan container untuk mengakses anotasi linguistik.

In [63]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("nama saya joni")


for token in doc:
    print(token)

nama
saya
joni


In [20]:
# Inisialisasi Doc - cara lain
from spacy.tokens import Doc

words = ["hello", "world", "!"]
spaces = [True, False, False] # artinya setelah hallo ada space (true), setelah world tidak ada space (False) dan setelah ! tidak ada space (False)
doc = Doc(nlp.vocab, words=words, spaces=spaces)

for token in doc:
    print(token, len(token))
    
print(doc)

hello 5
world 5
! 1
hello world!


In [25]:
doc = nlp("A Doc is a sequence of Token objects")

print(doc[0].text)
print(doc[-1].text)
span = doc[1:3]
print(span.text)

A
objects
Doc is


In [26]:
print([t.text for t in doc])

['A', 'Doc', 'is', 'a', 'sequence', 'of', 'Token', 'objects']


In [27]:
print(len(doc))

8


In [32]:
# set_extension

from spacy.tokens import Doc

city_getter = lambda doc: any(city in doc.text for city in ("New York", "Paris", "Berlin"))
Doc.set_extension("has_city", getter=city_getter, force=True)
doc = nlp("I like paris")
print(doc._.has_city)
extension = Doc.get_extension("has_city")
print(extension)

False
(None, None, <function <lambda> at 0x000002D45CE0A8B0>, None)


In [33]:
doc = nlp("I like New York")
span = doc.char_span(7, 15, label="GPE")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

New York 7 15 GPE


In [41]:
# set entity and its label
from spacy.tokens import Span

doc = nlp("Mr. Strong flew to New York on Saturday morning, He go with Mr. Smith")

doc.set_ents([Span(doc, 0, 2, "PERSON")])
ents = list(doc.ents)

for word in ents:
    print(word.text, word.label_)

Mr. Strong PERSON


In [58]:
# Doc similarity
import spacy
nlp = spacy.load("en_core_web_lg")
apples = nlp("i like apples")
oranges = nlp("i love oranges")

apples_oranges = apples.similarity(oranges)
oranges_apples = oranges.similarity(apples)

print("similarity dari apples - oranges",apples_oranges)
print("similarity dari oranges - apples",oranges_apples)

similarity dari apples - oranges 0.8885888838864333
similarity dari oranges - apples 0.8885888838864333


### Container Objects 3 - DocBin
Source: https://spacy.io/api/docbin

Digunakan untuk packing obyek Doc untuk serialisasi biner. Serialisasi yang digunakan adalah msgpack yang dikompres menggunakan gzip dengan format: 

'''
MSGPACK OBJECT STRUCTURE
{
    "version": str,           # DocBin version number
    "attrs": List[uint64],    # e.g. [TAG, HEAD, ENT_IOB, ENT_TYPE]
    "tokens": bytes,          # Serialized numpy uint64 array with the token data
    "spaces": bytes,          # Serialized numpy boolean array with spaces data
    "lengths": bytes,         # Serialized numpy int32 array with the doc lengths
    "strings": List[str]      # List of unique strings in the token data
}
'''

In [68]:
from spacy.tokens import DocBin

doc_bin = DocBin(attrs=["LEMMA"])
doc = nlp("The DocBin class lets you efficiently serialize the information from a collection of Doc objects.")

doc_bin.add(doc)
for token in doc:
    print(token)  
doc_bin.to_disk("./data.spacy")

doc_bin = DocBin().from_disk("./data.spacy")
for token in doc:
    print(token)

The
DocBin
class
lets
you
efficiently
serialize
the
information
from
a
collection
of
Doc
objects
.
The
DocBin
class
lets
you
efficiently
serialize
the
information
from
a
collection
of
Doc
objects
.


### Container Objects 4 - Lexeme

Source Lengkap: https://spacy.io/api/lexeme

Suatu entry di vocabulary tanpa konteks.

In [60]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love coffee for $5")
for word in doc:
    lexeme = doc.vocab[word.text]
    print(lexeme.text, lexeme.orth, lexeme.shape_, lexeme.prefix_, lexeme.suffix_,
            lexeme.is_alpha, lexeme.is_digit, lexeme.is_title, lexeme.lang_)
    
# orth => orthographic features, representasi text dalam hash
# shape_ => bentuk teks, karakter => X atau x, hanya 4 awal. Jika numerik: d

I 4690420944186131903 X I I True False True en
love 3702023516439754181 xxxx l ove True False False en
coffee 3197928453018144401 xxxx c fee True False False en
for 16037325823156266367 xxx f for True False False en
$ 11283501755624150392 $ $ $ False False False en
5 2090661578966068036 d 5 5 False True False en


### Container Objects Lain
Masih ada beberapa container objects:

Example: merupakan koleksi dari anotasi pelatihan. https://spacy.io/api/example
Span: potongan dari obyek Doc. https://spacy.io/api/span
SpanGroup: sekumpulan dari span, bisa diberi nama: https://spacy.io/api/spangroup
Token: https://spacy.io/api/token

### contoh lain DocBin

In [67]:
import spacy
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from spacy.tokens import DocBin

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# doc_bin = DocBin(attrs=["LEMMA"]) untuk lemma nya menggunakan sastrawi
doc = nlp(stemmer.stem("Berdasarkan contoh diatas tadi, maka dicobalah menggunakan sastrawi untuk merubah menjadi kata dasar dalam bahasa indonesia kemudian disimpan dalam DocBin"))

doc_bin.add(doc)
for token in doc:
    print(token)  
doc_bin.to_disk("./data_baru.spacy")

doc_bin = DocBin().from_disk("./data_baru.spacy")
for token in doc:
    print(token)


dasar
contoh
atas
tadi
maka
coba
guna
sastrawi
untuk
rubah
jadi
kata
dasar
dalam
bahasa
indonesia
kemudian
simpan
dalam
docbin
dasar
contoh
atas
tadi
maka
coba
guna
sastrawi
untuk
rubah
jadi
kata
dasar
dalam
bahasa
indonesia
kemudian
simpan
dalam
docbin
